In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!ls /content/drive/MyDrive/iNLP_Project/models

In [26]:
models_path = "/content/drive/MyDrive/iNLP_Project/models"

In [4]:
config={
    "learning_rate": 1e-5,
    "architecture": "xlmr",
    "dataset": "all",
}

In [5]:
%%capture
!pip install transformers
!pip install datasets
!pip install demoji
!pip install nltk
!pip install emoji
!pip install indic-nlp-library

In [6]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig, AutoModel, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from datasets import ClassLabel, Value
import torch
from tqdm.notebook import tqdm
import numpy as np
import seaborn as sns
import pickle
from collections import Counter
import sys, os
import demoji
import emoji
import string
import re

In [7]:
def remove_punctuation(text):
    return ''.join([c for c in text if c not in string.punctuation])

def remove_url(text):
    return re.sub(r'https?://\S+', ' ', text)

def remove_mention(text):
    return re.sub(r'@[A-Za-z0-9_]+', ' ', text)

def remove_hashtag(text):
    return re.sub(r'.*?(#\w+)|.+', ' ', text) 

def remove_emoji(text):
    # return emoji.demojize(text)
    return demoji.replace(text, ' ')

def normalize_hindi_script(text):
    from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
    factory=IndicNormalizerFactory()
    normalizer=factory.get_normalizer("mr",remove_nuktas=False)
    return normalizer.normalize(text)

def clean_row(text):
    return ' '.join(remove_punctuation(remove_url(remove_mention(remove_emoji(text)))).split()).lower()

def clean(df):
    df['clean_text'] = df.text.apply(clean_row)
    return df

In [8]:
test_df = pd.read_csv("/content/drive/MyDrive/iNLP_Project/Data/blind_test_pre_processed.csv")
test_df = clean(test_df)
print(test_df.columns)
test_df = test_df.drop(['text', 'text_id'], axis=1)
test_df.columns = ['id', 'frac_profane',	'polarity',	'text']
test_df.head()

Index(['Unnamed: 0', 'text_id', 'text', 'frac_profane', 'polarity',
       'clean_text'],
      dtype='object')


,id,frac_profane,polarity,text
0,0,0.0,0.713787,माणसाची सर्वात मोठी समस्या इतिहासाचे दाखले देत...
1,1,0.0,0.004368,मुद्दामहून थुकत बसला दोस्तांना बोलवून म्हणुन झ...
2,2,0.0,0.000637,तुझे नोटिफिकेशन ऑन केलेत परंतु नाव दिसलं शिव्य...
3,3,0.0,0.001381,पांडुरंग हरी वासुदेव हरी तु तुझ्या घरी माझ्या ...
4,4,0.0,-0.849870,भेंचोद जेव्हा कोरोना कोरोना चालू होतं तेव्हा आ...


In [9]:
master_df = pd.read_csv("/content/drive/MyDrive/iNLP_Project/Data/pre_processed.csv", index_col=0)
master_df = master_df.drop(['text_id'], axis=1)
master_df = clean(master_df)
master_df = master_df.drop(['text'], axis=1)

master_df.head()

,task_1,frac_profane,polarity,clean_text
0,NOT,0.0,0.001993,भारत 15 ऑगस्ट 1947 ला स्वतंत्र त्यानंतर तब्बल ...
1,NOT,0.0,-0.092219,स्वत ला हवा तसा बाइट किंवा प्रतिक्रिया घेण्यास...
2,NOT,0.0,0.002595,5 व्या नंबरची अर्थव्यवस्था भारताची जगात 2014 प...
3,HOF,0.2,-0.944961,च्यायला दुबईचा फोन पुडीच निघाली
4,HOF,0.0,-0.767109,ह्याला खरंतर कधीच आत टाकला पाहिजे पैसा स्टारडम...


In [10]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(master_df, test_size=0.1, stratify = master_df['task_1'])

train_df.columns = ['labels', 'frac_profane',	'polarity', 'text']
valid_df.columns = ['labels','frac_profane',	'polarity', 'text']

print(f"Train Shape: {train_df.shape}, Valid Shape: {valid_df.shape}")

train_df.to_csv('/content/drive/MyDrive/iNLP_Project/train.csv',columns=['labels', 'frac_profane',	'polarity', 'text'], index = False)
valid_df.to_csv('/content/drive/MyDrive/iNLP_Project/valid.csv',columns=['labels', 'frac_profane',	'polarity', 'text'], index = False)

display(train_df)
display(valid_df)


Train Shape: (1686, 4), Valid Shape: (188, 4)


,labels,frac_profane,polarity,text
140,NOT,0.000000,0.000817,माझा नवीन व्यवसाय मलाई मावा पेढे
1096,NOT,0.000000,0.000702,माझे कुटुंब माझी जबाबदारी मोहिमेवरून भाजप नेत्...
1304,NOT,0.000000,0.007005,नाट्यगृहे पुन्हा होण्याबाबत
268,HOF,0.071429,-0.239500,कारे मादरचोद नेरकर जास्तच माजलेला दिसतोय तू तु...
1469,NOT,0.062500,-0.987220,तुम्ही बहुतेक चुकून मोदींच्या ऐवजी उद्धव ठाकरे...
...,...,...,...,...
775,NOT,0.000000,-0.986596,आई सोबत कुठल्याही लग्नाला जाणे एकाच दिवसात खूप...
667,NOT,0.000000,0.005188,xbox series s स्वस्त एक्सबॉक्स गेमिंग कॉन्सोल
1332,NOT,0.000000,0.001626,मुंबईत सलग पाचव्या दिवशी दीड हजारापेक्षा रुग्ण...
327,NOT,0.090909,0.001764,च्यायला नेटिझन लै कार्टून लोकतीन चार वर्षांपूर...


,labels,frac_profane,polarity,text
796,NOT,0.000000,0.966067,एखाद्या नेत्याविरुद्ध बोलाल बंगल्यावर नेऊन बडव...
603,NOT,0.000000,0.098282,जाता हलकासा घाव दुरावणाऱ्या मनांना सुटणारा हाव
1459,NOT,0.000000,-0.976089,कोरोना निवारण मंत्र पांडूरंग हरी वासुदेव हरी त...
254,NOT,0.000000,0.001817,लऊडा or लवडा
1638,NOT,0.000000,0.001797,मराठी माणसांना आवाहन शिक्षणात दुसरी भाषा हिंदी...
...,...,...,...,...
1509,NOT,0.000000,-0.002106,qr स्कॅन जर कोणी पैसे घेत असेल सावध रहा कारण स...
1608,NOT,0.000000,-0.862443,कोण कुठली कंगना तीच घर पाडलं सगळा देश उठला इथे...
1435,NOT,0.000000,-0.956789,माझ्या छातीमध्ये कळ गरीबांचा तांदुळ खाऊन माझ्य...
800,NOT,0.000000,0.002979,चला भाजप ची b टी


In [11]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
TEST_BATCH_SIZE = 8
LEARNING_RATE = 1e-05
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base', truncation=True)

In [12]:
labels = ClassLabel(num_classes = 2, names = ['HOF', 'NOT'])

In [13]:
class HindiData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, isTest = False):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        # self.features = dataframe[['frac_profane',	'polarity']].values
        self.max_len = max_len
        self.isTest = isTest
        if not self.isTest:
            self.targets = self.data.labels
        # print(self.features)

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        if not self.isTest:
            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
                'targets': torch.tensor(labels.str2int(self.targets[index]), dtype=torch.float)
                # 'features': torch.tensor(self.features[index],dtype=torch.float)
            }
        else:
            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
                # 'features': torch.tensor(self.features[index],dtype=torch.float)
            }

In [14]:
train_data=train_df.reset_index(drop=True)
valid_data = valid_df.reset_index(drop=True)

print("TRAIN Dataset: {}".format(train_data.shape))
print("VALID Dataset: {}".format(valid_data.shape))

training_set = HindiData(train_data, tokenizer, MAX_LEN, False)
valid_set = HindiData(valid_data, tokenizer, MAX_LEN, False)

TRAIN Dataset: (1686, 4)
VALID Dataset: (188, 4)


In [15]:
test_data=test_df.reset_index(drop=True)
print("TEST Dataset: {}".format(test_data.shape))
testing_set = HindiData(test_data, tokenizer, MAX_LEN, True)

TEST Dataset: (625, 4)


In [16]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

valid_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
valid_loader = DataLoader(valid_set, **valid_params)

In [17]:
test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

testing_loader = DataLoader(testing_set, **test_params)

In [18]:
from torch import nn

class CustomXLMRModel(torch.nn.Module):
    def __init__(self,num_labels=2):
        super(CustomXLMRModel, self).__init__()
        self.num_labels = num_labels
        self.xlmr = AutoModel.from_pretrained("xlm-roberta-base", num_labels = num_labels)
        # self.xlmr = RobertaModel.from_pretrained("roberta-base")
        ### New layers:
        self.conv_layer = torch.nn.Conv1d(768, 1, kernel_size=3)
        
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(12, 2)
        self.dropout = nn.Dropout(0.4)

    def forward(self, ids, mask, token_type_ids):
        outputs = self.xlmr(ids, attention_mask=mask, token_type_ids=token_type_ids, output_hidden_states=True)
        
        last_hidden_state, pooler_output, allhidden = outputs[0], outputs[1], outputs[2]
        
        # print(len(allhidden))
        # print(allhidden[0].size())
        # print(allhidden[1].size())

        feature_vec = []

        for i, hstate in enumerate(allhidden[1:]):
            
            # print("For hidden state:", i+1)
            # print("original shape", hstate.size())

            hstate = hstate.permute(0, 2, 1)
            # print("transposed shape", hstate.size())

            conv_out = F.relu(self.conv_layer(hstate))
            conv_out = conv_out.to(device)

            # print("shape after conv:", conv_out.size())
            
            max_out = F.max_pool1d(conv_out, kernel_size=conv_out.shape[2])
            # print("shape after max pool:", max_out.size())

            feature_vec.append(max_out)

        result = torch.cat(feature_vec, dim=1).squeeze(dim=2)
        #print("result shape:", result.size())
        #print("feature shape:", features.size())
        # result = torch.cat((result,features),1)
        result = result.to(device)
        logits = self.fc(self.dropout(result))
        logits = logits.to(device)

        return logits

In [19]:
torch.cuda.empty_cache()
# model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels = 2)
model = CustomXLMRModel()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CustomXLMRModel(
  (xlmr): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
           

In [20]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
# loss_function = torch.nn.BCELoss()
# loss_function = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-5, weight_decay = 1e-5)
# optimizer = torch.optim.Adadelta(model.parameters(),
#                                lr=2e-5,
#                                rho=0.95)

In [21]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [22]:
from sklearn.metrics import (accuracy_score, confusion_matrix,
                             precision_recall_fscore_support)

In [23]:
best_val_loss = 0
best_val_acc = 0

def train(epoch):

    #------------------------------------------------#
    #                TRAIN BLOCK                     #
    #------------------------------------------------#
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    y_pred = []
    y_true = []
    y_pred_val = []
    y_true_val = []
    model.train()


    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        # features = data['features'].to(device, dtype = torch.float32)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        # big_idx = outputs.data
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        y_pred.extend(big_idx.tolist())
        y_true.extend(targets.cpu().numpy())
        
        if _%10==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    #------------------------------------------------#
    #                VALID BLOCK                     #
    #------------------------------------------------#

    val_loss = 0
    n_correct_val = 0
    nb_val_steps = 0
    nb_val_examples = 0
    model.eval()

    for _,data in tqdm(enumerate(valid_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        # features = data['features'].to(device, dtype = torch.float32)

        outputs = model(ids, mask, token_type_ids)
        vloss = loss_function(outputs, targets)
        val_loss += vloss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct_val += calcuate_accuracy(big_idx, targets)

        nb_val_steps += 1
        nb_val_examples+=targets.size(0)
        
        if _%10==0:
            val_loss_step = val_loss/nb_val_steps
            val_accu_step = (n_correct_val*100)/nb_val_examples 
        y_pred_val.extend(big_idx.tolist())
        y_true_val.extend(targets.cpu().numpy())


    #------------------------------------------------#
    #                LOG BLOCK                       #
    #------------------------------------------------#

    print(f'The Total Train Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    print(f'The Total Val Accuracy for Epoch {epoch}: {(n_correct_val*100)/nb_val_examples}')
    epoch_loss_val = tr_loss/nb_val_steps
    epoch_accu_val = (n_correct_val*100)/nb_val_examples
    print(f"Val Loss Epoch: {epoch_loss_val}")
    print(f"Val Accuracy Epoch: {epoch_accu_val}")

    print("--------------------------------------------------")

    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    epoch_accu = accuracy_score(y_true, y_pred)
    print(f"Train Stats -- Accu: {epoch_accu}, Prec: {precision}, Recall: {recall}, F1: {f1}")

    precision, recall, f1, _ = precision_recall_fscore_support(y_true_val, y_pred_val, average='macro')
    epoch_accu = accuracy_score(y_true_val, y_pred_val)
    print(f"Val Stats -- Accu: {epoch_accu}, Prec: {precision}, Recall: {recall}, F1: {f1}")

    print("\n")

    #------------------------------------------------#
    #                SAVE  BLOCK                     #
    #------------------------------------------------#

    global best_val_loss
    global best_val_acc

    if epoch == 0:
        torch.save(model, os.path.join(models_path, 'best_xlmr_mr_model_bce_taska4.pt'))
        best_val_loss = epoch_loss_val
        best_val_acc = epoch_accu_val

    elif epoch_accu_val > best_val_acc:
        best_val_acc = epoch_accu_val
        torch.save(model, os.path.join(models_path, 'best_xlmr_mr_model_bce_taska4.pt'))

    # elif epoch_loss_val < best_val_loss:
    #     best_val_loss = epoch_loss_val
    #     torch.save(model, os.path.join(models_path, 'best_xlmr_hi_model.pt'))

    print("Best loss so far", best_val_loss)
    print("Best Accu so far", best_val_acc)

    return 

In [24]:
EPOCHS = 15
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


0it [00:00, ?it/s]

The Total Train Accuracy for Epoch 0: 59.66785290628707
Training Loss Epoch: 0.6738228573335856
Training Accuracy Epoch: 59.66785290628707
The Total Val Accuracy for Epoch 0: 59.57446808510638
Val Loss Epoch: 3.0250345297316286
Val Accuracy Epoch: 59.57446808510638
--------------------------------------------------
Train Stats -- Accu: 0.5966785290628707, Prec: 0.5120137669777326, Recall: 0.5079700506307389, F1: 0.49108032237616595
Val Stats -- Accu: 0.5957446808510638, Prec: 0.43343653250773995, Recall: 0.4761317380041939, F1: 0.41748206131767773


Best loss so far 3.0250345297316286
Best Accu so far 59.57446808510638


0it [00:00, ?it/s]

...

The Total Train Accuracy for Epoch 13: 94.48398576512456
Training Loss Epoch: 0.18946356694898192
Training Accuracy Epoch: 94.48398576512456
The Total Val Accuracy for Epoch 13: 87.76595744680851
Val Loss Epoch: 0.8505704814092593
Val Accuracy Epoch: 87.76595744680851
--------------------------------------------------
Train Stats -- Accu: 0.9448398576512456, Prec: 0.9432270776707347, Recall: 0.9360526412573096, F1: 0.9394505764818869
Val Stats -- Accu: 0.8776595744680851, Prec: 0.8718637992831542, Recall: 0.8583323054150733, F1: 0.8642940087248533


Best loss so far 3.0250345297316286
Best Accu so far 89.8936170212766


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


0it [00:00, ?it/s]

The Total Train Accuracy for Epoch 14: 95.37366548042705
Training Loss Epoch: 0.16829832960626384
Training Accuracy Epoch: 95.37366548042705
The Total Val Accuracy for Epoch 14: 87.76595744680851
Val Loss Epoch: 0.7555520754664186
Val Accuracy Epoch: 87.76595744680851
--------------------------------------------------
Train Stats -- Accu: 0.9537366548042705, Prec: 0.9528938816674137, Recall: 0.9459259418175576, F1: 0.9492363135530412
Val Stats -- Accu: 0.8776595744680851, Prec: 0.8694556451612903, Recall: 0.8616627605772789, F1: 0.8652749649478111


Best loss so far 3.0250345297316286
Best Accu so far 89.8936170212766
